In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed ,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

In [2]:
df=pd.read_csv("../data/processed/data_v1.csv")


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221948 entries, 0 to 1221947
Data columns (total 17 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   timestamp        1221948 non-null  object 
 1   TP2              1221948 non-null  float64
 2   TP3              1221948 non-null  float64
 3   H1               1221948 non-null  float64
 4   DV_pressure      1221948 non-null  float64
 5   Reservoirs       1221948 non-null  float64
 6   Oil_temperature  1221948 non-null  float64
 7   Motor_current    1221948 non-null  float64
 8   COMP             1221948 non-null  float64
 9   DV_eletric       1221948 non-null  float64
 10  Towers           1221948 non-null  float64
 11  MPG              1221948 non-null  float64
 12  LPS              1221948 non-null  float64
 13  Pressure_switch  1221948 non-null  float64
 14  Oil_level        1221948 non-null  float64
 15  Caudal_impulses  1221948 non-null  float64
 16  Air_Leak         1

In [4]:
df["Air_Leak"].value_counts()

Air_Leak
0    1196741
1      25207
Name: count, dtype: int64

In [5]:
y=df["Air_Leak"]


In [6]:
df['timestamp']=pd.to_datetime(df["timestamp"])

In [7]:
df.drop(columns=["Reservoirs","COMP","Caudal_impulses","Pressure_switch","H1"],inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221948 entries, 0 to 1221947
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   timestamp        1221948 non-null  datetime64[ns]
 1   TP2              1221948 non-null  float64       
 2   TP3              1221948 non-null  float64       
 3   DV_pressure      1221948 non-null  float64       
 4   Oil_temperature  1221948 non-null  float64       
 5   Motor_current    1221948 non-null  float64       
 6   DV_eletric       1221948 non-null  float64       
 7   Towers           1221948 non-null  float64       
 8   MPG              1221948 non-null  float64       
 9   LPS              1221948 non-null  float64       
 10  Oil_level        1221948 non-null  float64       
 11  Air_Leak         1221948 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 111.9 MB


## timeseries Analysis


In [9]:
df.set_index('timestamp', inplace=True)


In [10]:
df.sort_index(inplace=True)


In [11]:
df.head()

,TP2,TP3,DV_pressure,Oil_temperature,Motor_current,DV_eletric,Towers,MPG,LPS,Oil_level,Air_Leak
timestamp,,,,,,,,,,,
2020-02-01 00:00:00,-0.012,9.358,-0.024,53.600,0.0400,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:10,-0.014,9.348,-0.022,53.675,0.0400,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:19,-0.012,9.338,-0.022,53.600,0.0425,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:29,-0.012,9.328,-0.022,53.425,0.0400,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:39,-0.012,9.318,-0.022,53.475,0.0400,0.0,1.0,1.0,0.0,1.0,0


In [12]:

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df.drop(columns=['Air_Leak']))  
df_scaled = pd.DataFrame(scaled_features, index=df.index, columns=df.columns[:-1])
df_scaled['Air_Leak'] = df['Air_Leak'].values
df_scaled.shape

(1221948, 11)

In [13]:
df_scaled.head()

,TP2,TP3,DV_pressure,Oil_temperature,Motor_current,DV_eletric,Towers,MPG,LPS,Oil_level,Air_Leak
timestamp,,,,,,,,,,,
2020-02-01 00:00:00,0.001868,0.901379,0.000810,0.564046,0.002156,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:10,0.001681,0.900334,0.001013,0.565153,0.002156,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:19,0.001868,0.899290,0.001013,0.564046,0.002426,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:29,0.001868,0.898245,0.001013,0.561462,0.002156,0.0,1.0,1.0,0.0,1.0,0
2020-02-01 00:00:39,0.001868,0.897200,0.001013,0.562200,0.002156,0.0,1.0,1.0,0.0,1.0,0


In [14]:
def create_sequences(data, labels, sequence_length=10):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(labels[i+sequence_length])  # Supervised for fine-tuning
    return np.array(X), np.array(y)

X, y = create_sequences(df_scaled.drop(columns=['Air_Leak']).values, df_scaled['Air_Leak'].values)

In [15]:
X.shape,y.shape 

((1221938, 10, 10), (1221938,))

In [16]:
X

array([[[1.86776242e-03, 9.01379022e-01, 8.10044552e-04, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.68098618e-03, 9.00334308e-01, 1.01255569e-03, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.86776242e-03, 8.99289595e-01, 1.01255569e-03, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        ...,
        [1.68098618e-03, 8.93857083e-01, 8.10044552e-04, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.86776242e-03, 8.92812369e-01, 1.01255569e-03, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.86776242e-03, 8.91558713e-01, 1.01255569e-03, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00]],

       [[1.68098618e-03, 9.00334308e-01, 1.01255569e-03, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.86776242e-03, 8.99289595e-01, 1.01255569e-03, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.86776242e-03, 

In [17]:
def create_semi_supervised_lstm_autoencoder(input_shape, l2_lambda=0.001, dropout_rate=0.4):
    # Encoder
    inputs = Input(shape=input_shape)
    encoded = LSTM(64, activation='relu', return_sequences=True, kernel_regularizer=l2(l2_lambda))(inputs)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = LSTM(32, activation='relu', return_sequences=False, kernel_regularizer=l2(l2_lambda))(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    
    # Decoder
    decoded = RepeatVector(input_shape[0])(encoded)
    decoded = LSTM(32, activation='relu', return_sequences=True, kernel_regularizer=l2(l2_lambda))(decoded)
    decoded = Dropout(0.4)(decoded)

    decoded = LSTM(64, activation='relu', return_sequences=True, kernel_regularizer=l2(l2_lambda))(decoded)
    
    # Reconstruction output
    reconstruction = TimeDistributed(Dense(input_shape[1]), name="reconstruction")(decoded)
    
    # Classification Head
    classification = Dense(1, activation='sigmoid', name="classification")(encoded)

    # Define the model
    autoencoder = Model(inputs, outputs=[reconstruction, classification])
    autoencoder.compile(
        optimizer=Adam(learning_rate=0.0005), 
        loss={"reconstruction": "mean_squared_error", "classification": "binary_crossentropy"},
        loss_weights={"reconstruction": 0.5, "classification": 0.5}
    )
    
    return autoencoder


In [18]:
normal_data = X[y == 0]

autoencoder = create_semi_supervised_lstm_autoencoder(normal_data[0].shape)
autoencoder.fit(
    normal_data, [normal_data, np.zeros(len(normal_data))],
    epochs=5, batch_size=32, validation_split=0.2
)


Epoch 1/5
29919/29919 ━━━━━━━━━━━━━━━━━━━━ 418s 14ms/step - loss: 0.0214 - val_loss: 0.0331
Epoch 2/5
29919/29919 ━━━━━━━━━━━━━━━━━━━━ 371s 12ms/step - loss: 0.0055 - val_loss: 0.0320
Epoch 3/5
29919/29919 ━━━━━━━━━━━━━━━━━━━━ 348s 12ms/step - loss: 0.0057 - val_loss: 0.0313
Epoch 4/5
29919/29919 ━━━━━━━━━━━━━━━━━━━━ 350s 12ms/step - loss: 0.0054 - val_loss: 0.0327
Epoch 5/5
29919/29919 ━━━━━━━━━━━━━━━━━━━━ 352s 12ms/step - loss: 0.0059 - val_loss: 0.0321


In [19]:
autoencoder.fit(
    X, [X, y],
    epochs=5, batch_size=32, validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)


Epoch 1/5
30549/30549 ━━━━━━━━━━━━━━━━━━━━ 363s 12ms/step - loss: 0.0763 - val_loss: 0.0348
Epoch 2/5
30549/30549 ━━━━━━━━━━━━━━━━━━━━ 358s 12ms/step - loss: 0.0359 - val_loss: 0.0346
Epoch 3/5
30549/30549 ━━━━━━━━━━━━━━━━━━━━ 358s 12ms/step - loss: 0.0289 - val_loss: 0.0329
Epoch 4/5
30549/30549 ━━━━━━━━━━━━━━━━━━━━ 356s 12ms/step - loss: 0.0206 - val_loss: 0.0336
Epoch 5/5
30549/30549 ━━━━━━━━━━━━━━━━━━━━ 362s 12ms/step - loss: 0.0177 - val_loss: 0.0330


In [24]:

# Predict using the classification head
_, predictions = autoencoder.predict(X)
predicted_anomalies = (predictions > 0.5).astype(int)

# Evaluate metrics
accuracy = accuracy_score(y, predicted_anomalies)
precision = precision_score(y, predicted_anomalies)
recall = recall_score(y, predicted_anomalies)
f1 = f1_score(y, predicted_anomalies)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


38186/38186 ━━━━━━━━━━━━━━━━━━━━ 158s 4ms/step
Accuracy: 0.9946
Precision: 0.8147
Recall: 0.9536
F1 Score: 0.8787


In [25]:
conf_matrix=confusion_matrix(y, predicted_anomalies)
print(conf_matrix)


[[1191263    5468]
 [   1170   24037]]


In [29]:
def preprocess(df):
    test_data=pd.read_csv(f"../data/processed/{df}.csv")
    test_data['timestamp']=pd.to_datetime(test_data["timestamp"])
    y=test_data["Air_Leak"]
    test_data.drop(columns=["Air_Leak","Reservoirs","COMP","Caudal_impulses","Pressure_switch","H1"],inplace=True)
    test_data.set_index('timestamp', inplace=True)

    test_data.sort_index(inplace=True)
 
    print(test_data.columns)

    
    scaled_data = scaler.transform(test_data)  
    print(scaled_data.shape)
    new_data_sequences,_ = create_sequences(scaled_data,labels=np.zeros(len(scaled_data)) ,sequence_length=10)
    return new_data_sequences,y

preprocess("data_v2")
        


Index(['TP2', 'TP3', 'DV_pressure', 'Oil_temperature', 'Motor_current',
       'DV_eletric', 'Towers', 'MPG', 'LPS', 'Oil_level'],
      dtype='object')
(295000, 10)


(array([[[0.76671647, 0.75470121, 0.19947347, ..., 0.        ,
          0.        , 1.        ],
         [0.56817333, 0.75428333, 0.13345484, ..., 0.        ,
          0.        , 1.        ],
         [0.71441913, 0.75198496, 0.181855  , ..., 0.        ,
          0.        , 1.        ],
         ...,
         [0.60870377, 0.75323861, 0.1472256 , ..., 0.        ,
          0.        , 1.        ],
         [0.73907359, 0.7507313 , 0.19015796, ..., 0.        ,
          0.        , 1.        ],
         [0.76447516, 0.75094024, 0.19866343, ..., 0.        ,
          0.        , 1.        ]],
 
        [[0.56817333, 0.75428333, 0.13345484, ..., 0.        ,
          0.        , 1.        ],
         [0.71441913, 0.75198496, 0.181855  , ..., 0.        ,
          0.        , 1.        ],
         [0.76092641, 0.75094024, 0.19744836, ..., 0.        ,
          0.        , 1.        ],
         ...,
         [0.73907359, 0.7507313 , 0.19015796, ..., 0.        ,
          0.        , 1.

In [ ]:
new_seq,true_labels=preprocess("data_v2")
_,predictions = autoencoder.predict(new_seq)




Index(['TP2', 'TP3', 'DV_pressure', 'Oil_temperature', 'Motor_current',
       'DV_eletric', 'Towers', 'MPG', 'LPS', 'Oil_level'],
      dtype='object')
(295000, 10)
9219/9219 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step


In [35]:
# Evaluate metrics
predicted_anomalies = (predictions > 0.5).astype(int)
true_labels=true_labels[:294990]
accuracy = accuracy_score(true_labels, predicted_anomalies)
precision = precision_score(true_labels, predicted_anomalies)
recall = recall_score(true_labels, predicted_anomalies)
f1 = f1_score(true_labels, predicted_anomalies)
print(f"Accuractrue_labels: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuractrue_labels: 0.9939
Precision: 0.9489
Recall: 0.6575
F1 Score: 0.7768


In [37]:
conf_matrix=confusion_matrix(true_labels, predicted_anomalies)
print(conf_matrix)


[[290075    168]
 [  1626   3121]]
